In [4]:
# Requiere: vedo, numpy, pandas, trimesh, open3d
import vedo
import pandas as pd
import numpy as np
import open3d as o3d
import trimesh
import os

# --- Función para cargar el CSV ---
def cargar_datos_csv(ruta_csv):
    return pd.read_csv(ruta_csv)

# --- Crear primitiva en Vedo desde fila de datos ---
def crear_primitiva(fila):
    x, y, z = fila['x'], fila['y'], fila['z']
    tipo = fila['tipo'].lower()
    color = fila['color']
    escala = fila['escala']

    if tipo == 'cubo':
        return vedo.Cube(pos=(x, y, z), side=escala, c=color)
    elif tipo == 'esfera':
        return vedo.Sphere(pos=(x, y, z), r=escala, c=color)
    elif tipo == 'cilindro':
        return vedo.Cylinder(pos=(x, y, z), r=escala/2, height=escala*1.5, c=color)
    else:
        return None

# --- Generar lista de objetos 3D ---
def generar_objetos(df):
    return [crear_primitiva(fila) for _, fila in df.iterrows() if crear_primitiva(fila)]

# --- Visualizar escena en ventana interactiva ---
def visualizar_escena(objetos):
    escena = vedo.Plotter(title="Escena Paramétrica 3D", axes=1)
    escena.show(objetos, interactive=True)

# --- Exportar escena a 3 formatos ---
def exportar_escena(objetos, nombre_salida='escena_parametrica'):
    os.makedirs("../resultados/python/modelos", exist_ok=True)
    ruta_base = os.path.join("../resultados/python/modelos", nombre_salida)

    # Unificar todos los objetos en una sola malla
    mesh_vedo = vedo.merge(*objetos)
    vedo.write(mesh_vedo, f"{ruta_base}.obj")
    vedo.write(mesh_vedo, f"{ruta_base}.stl")

    # Exportar como GLB/GLTF con Open3D
    triangles = vedo.Mesh(mesh_vedo).triangulate()
    puntos = o3d.utility.Vector3dVector(triangles.points)
    # Extraer caras de la malla
    caras_np = np.array(triangles.cells, dtype=np.int32)

    # Asegurar que tiene forma (N, 3)
    if caras_np.ndim == 1:
        caras_np = caras_np.reshape(-1, 3)

    # Crear la malla Open3D
    caras = o3d.utility.Vector3iVector(caras_np)
    malla_o3d = o3d.geometry.TriangleMesh(vertices=puntos, triangles=caras)
    o3d.io.write_triangle_mesh(f"{ruta_base}.glb", malla_o3d)

ruta_csv = "../datos/datos_escena.csv"
datos = cargar_datos_csv(ruta_csv)
objetos = generar_objetos(datos)
visualizar_escena(objetos)
exportar_escena(objetos)



ModuleNotFoundError: No module named 'open3d'

In [6]:
%pip install open3d==0.17.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement open3d==0.17.0 (from versions: none)
ERROR: No matching distribution found for open3d==0.17.0
